In [ ]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Încărcare date [cite: 11]
try:
    df = pd.read_csv('date_colesterol.csv')
except FileNotFoundError:
    print("Eroare: Fișierul date_colesterol.csv nu a fost găsit.")

print(df.head())

# Extragem datele și le standardizăm (important pentru modele mixte)
# Verifică numele coloanelor din print-ul de mai sus!
X = df['exercise'].values
Y = df['cholesterol'].values

# Standardizare (Z-score)
X_mean, X_std = X.mean(), X.std()
Y_mean, Y_std = Y.mean(), Y.std()
X_s = (X - X_mean) / X_std
Y_s = (Y - Y_mean) / Y_std

compare_dict = {}
# Lista de clustere cerută [cite: 12]
K_list = [3, 4, 5]

# 1. Estimare ponderi și coeficienți
for K in K_list:
    print(f"Running Mixture Model for K={K}...")
    with pm.Model() as model_mix:
        # Ponderile subpopulațiilor
        weights = pm.Dirichlet('weights', a=np.ones(K))
        
        # Parametrii polinomului pentru fiecare cluster k
        # Model polinomial: mu = alpha + beta*t + gamma*t^2 [cite: 10]
        alpha = pm.Normal('alpha', 0, 5, shape=K)
        beta = pm.Normal('beta', 0, 5, shape=K)
        gamma = pm.Normal('gamma', 0, 5, shape=K)
        sigma = pm.HalfNormal('sigma', 1, shape=K)
        
        # Calculăm media pentru fiecare componentă
        # Broadcasting: (K, 1) vs (1, N) -> (K, N)
        mu = alpha[:, None] + beta[:, None] * X_s + gamma[:, None] * (X_s**2)
        
        # Likelihood Mixture [cite: 6]
        y_obs = pm.NormalMixture('y_obs', w=weights, mu=mu, sigma=sigma[:, None], observed=Y_s)
        
        idata = pm.sample(1000, return_inferencedata=True, progressbar=True)
        compare_dict[f'K={K}'] = idata

# 2. Comparație modele folosind WAIC [cite: 13]
comp_df = az.compare(compare_dict, ic="waic")
print("Rezultate comparație modele:")
print(comp_df)

az.plot_compare(comp_df)
plt.title("Comparatie modele (WAIC)")
plt.show()

best_model = comp_df.index[0]
print(f"Conform criteriului WAIC, cel mai bun model este: {best_model}")